In [ ]:
# 시작하기 전에 mount하기
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_sas('/content/drive/MyDrive/hn18_all.sas7bdat', format = 'sas7bdat')
# hn18_all.sas7bdat 파일의 경로 복사하기 (국민건강영양조사 데이터셋 사용)

In [ ]:
# 원시 데이터 가공하기

selected_data = data[['DI1_dg', 'sex', 'age', 'HE_sbp', 'HE_dbp', 'HE_BMI', 'HE_PLS']]

#흡연 여부
selected_data['sm_present'] = ((data['BS1_1'].isin([1, 2]) & data['BS3_1'].isin([1, 2, 3])) | (data['BS1_1'] == 3)).astype(int)
selected_data.loc[data['BS1_1'] == 2, 'sm_present'] = (data['BS3_1'].isin([1, 2])).astype(int)

#pa_walk : 주5회 걷기 운동을 30분이상 실천한 여부
data['pa_hb30_1'] = data['BE3_32'] * 60 + data['BE3_33']
selected_data['pa_walk'] = (data['BE3_31'].isin([6, 7, 8]) & (data['pa_hb30_1'] >= 30)).astype(int)

#수면시간
data['sleep'] = np.where((data['BP16_11'].isin([88,99]) | data['BP16_12'].isin([88,99]) | data['BP16_13'].isin([88,99]) | data['BP16_14'].isin([88,99])),
                         None, (data['BP16_13']+data['BP16_14']/60 - data['BP16_11']+data['BP16_12']/60 + 24)%24)
selected_data['total_sleep'] = data['sleep']

In [ ]:
# dropna 함수를 사용해 결측치가 포함된 행을 제거

print(selected_data.shape)
selected_data = selected_data.dropna()
print(selected_data.shape)

In [ ]:
X = selected_data.drop('DI1_dg', axis=1).dropna()
y = selected_data.loc[X.index, 'DI1_dg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.astype('float')
X_test = X_test.astype('float')
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [ ]:
print(X_train.dtypes)
print('-')
print('고혈압 여부: ' + str(y_train.dtypes))

In [ ]:
print('나이')
print(max(X_train['age'].values))
print(min(X_train['age'].values))
print('수축기 혈압')
print(max(X_train['HE_sbp'].values))
print(min(X_train['HE_sbp'].values))
print('확장기 혈압')
print(max(X_train['HE_dbp'].values))
print(min(X_train['HE_dbp'].values))
print('체질량 지수')
print(max(X_train['HE_BMI'].values))
print(min(X_train['HE_BMI'].values))
print('15초 맥박수')
print(max(X_train['HE_PLS'].values))
print(min(X_train['HE_PLS'].values))
print('수면 시간')
print(max(X_train['total_sleep'].values))
print(min(X_train['total_sleep'].values))
print('흡연 여부')
print(max(X_train['sm_present'].values))
print(min(X_train['sm_present'].values))
print('유산소 운동 여부')
print(max(X_train['pa_walk'].values))
print(min(X_train['pa_walk'].values))

In [ ]:
# 데이터 정규화하기 (0~1 사이의 값으로)
def normalize_column(column):
    return (column - column.min()) / (column.max() - column.min())

norm_x_train = X_train.apply(normalize_column)
norm_x_test = X_test.apply(normalize_column)

In [ ]:
print(norm_x_test)

In [ ]:
# mnist 숫자 인식의 모델 구조 참고해서 구성해보기
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
mlp = Sequential()
mlp.add(Dense(units=512, activation='tanh', input_shape=(9,), name='one'))
mlp.add(Dense(units=1, activation='sigmoid', name='two'))

mlp.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])
history = mlp.fit(norm_x_train, y_train, batch_size=128, epochs=50, validation_data=(norm_x_test, y_test))

res = mlp.evaluate(norm_x_test, y_test)
print('정확률=', res[1]*100)
# 지금은 정확률이 0.63 정도로 SVM보다 못한 성능을 보여준다.
# loss 값이 왜 저렇게 음의 방향으로 커지는 지에 대해서도 조사가 필요해 보인다.
# 모델 구조 개선이나 하이퍼파라미터 변경, 추가 데이터셋이 필요하다. (추후 할 예정)

In [ ]:
from keras import models

# 모델 저장
# mlp.save('/content/drive/MyDrive/my_model')

# 고혈압 예측 확률(추정)
a = mlp.predict(norm_x_test[:1])[0][0]
print(a)
print(type(a))